In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import talib
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
def TechnicalStrategy(data_price,talib_func):

    '''
    df_price:价格数据；
    talib_func:公式
    '''
    # 2.0 get基础数据并计算
    data_price = data_price.copy()
    data_price.index = data_price.index.strftime('%Y%m%d')
    # 使用talib算法计算技术指标
    if   talib_func.__name__ == 'MIDPRICE':
        data_price['sma'] = talib_func(data_price['high'], data_price['low'],timeperiod=6)
        data_price['lma'] = talib_func(data_price['high'], data_price['low'],timeperiod=30)
    elif talib_func.__name__ == 'ADX':
        data_price['sma'] = talib_func(data_price['high'], data_price['low'],data_price['close'],timeperiod=6)
        data_price['lma'] = talib_func(data_price['high'], data_price['low'],data_price['close'],timeperiod=30)
    elif talib_func.__name__ == 'AROON' :
        data_price['sma'] = talib_func(data_price['high'], data_price['low'],timeperiod=6)[0]
        data_price['lma'] = talib_func(data_price['high'], data_price['low'],timeperiod=30)[0]
    else:
        data_price['sma'] = talib_func(data_price['close'], timeperiod=6)
        data_price['lma'] = talib_func(data_price['close'], timeperiod=30)
    # 2.2 均线策略的交易记录
    Buy = []  # 保存买入记录
    Sell = []  # 保存卖出记录
    price_in = 1  # 初始买入价设置为1
    data_price['position'] = 0
    data_price['flag'] = 0.0  # 记录买卖
    for i in range(max(1, 30), data_price.shape[0]):
        data_price = data_price.copy()
        if (data_price['position'][i - 1] == 0) and (data_price['sma'][i - 1] < data_price['lma'][i - 1]) and (
                data_price['sma'][i] > data_price['lma'][i]):
            data_price['flag'][i] = 1  # 记录买入还是卖出，1是买入
            data_price['position'][i] = 1  # 仓位记录为1，表示有1手仓位
            date_in = data_price.index[i]  # 记录买入的时间 年-月-日
            price_in = data_price['close'][i]  # 记录买入的价格，这里是以收盘价买入
            entry_index = i
            Buy.append([date_in, price_in, '金叉买入'])  # 把买入记录保存到Buy列表里
        elif (data_price['position'][i - 1] == 1) & (data_price['sma'][i - 1] > data_price['lma'][i - 1]) & (
                data_price['sma'][i] < data_price['lma'][i]):
            data_price['flag'][i] = -1  # 记录买入还是卖出，1是买入
            data_price['position'][i] = 0  # 仓位记录为0，表示没有仓位了
            date_out = data_price.index[i]  # 记录卖出的时间 年-月-日
            price_out = data_price['close'][i]  # 记录卖出的价格，这里是以收盘价卖出
            Sell.append([date_out, price_out, '死叉卖出'])  # 把卖出记录保存到Sell列表里
        else:
            data_price['position'][i] = data_price['position'][i - 1]
    p1 = pd.DataFrame(Buy, columns=['买入日期', '买入价格', '备注'])
    p2 = pd.DataFrame(Sell, columns=['卖出日期', '卖出价格', '备注'])
    transactions = pd.concat([p1, p2], axis=1)  # p1和p2合并，axis=1表示以水平方向合并
    data_price = data_price.iloc[30:, :]
    data_price['position'] = data_price['position'].shift(1).fillna(0)  # 思考：为什么data_price['position']要向后移动一个单位shift(1)？
    data_price['ret'] = data_price.close.pct_change(1).fillna(0)
    return (data_price.ret * data_price.position).mean() / (data_price.ret * data_price.position).std() * np.sqrt(252)

def handle_data(df):
    df.dropna(inplace = True)
    df = df[['日期', '开盘价(元)', '最高价(元)', '最低价(元)', '收盘价(元)','成交量(股)']]
    df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp',inplace  = True)
    return df

In [8]:
# homework1
'''pandasd的pct_change函数计算的是当前位置和前一位置之间变化的百分比，position后移一位的话才可以和收益率对齐 '''

'pandasd的pct_change函数计算的是当前位置和前一位置之间变化的百分比，position后移一位的话才可以和收益率对齐 '

In [88]:
# homework2
result = pd.DataFrame()
file_name = os.listdir(r'D:\data\raw_data\十种大宗商品指数文件')
file_name = [i for i in file_name if not i.startswith('~')]
talib_func = [talib.MA,talib.DEMA,talib.EMA,talib.KAMA,talib.MIDPRICE,talib.T3,talib.TEMA,talib.WMA,talib.ADX,talib.AROON]
for i in file_name:
    df = handle_data(pd.read_excel(f'D:/data/raw_data/十种大宗商品指数文件/{i}'))
    for j in talib_func:
        result.at[i[:-5],j.__name__] = TechnicalStrategy(df,j)
        print(i[:-5],j.__name__)

pta指数 MA
pta指数 DEMA
pta指数 EMA
pta指数 KAMA
pta指数 MIDPRICE
pta指数 T3
pta指数 TEMA
pta指数 WMA
pta指数 ADX
pta指数 AROON
塑料指数 MA
塑料指数 DEMA
塑料指数 EMA
塑料指数 KAMA
塑料指数 MIDPRICE
塑料指数 T3
塑料指数 TEMA
塑料指数 WMA
塑料指数 ADX
塑料指数 AROON
橡胶指数 MA
橡胶指数 DEMA
橡胶指数 EMA
橡胶指数 KAMA
橡胶指数 MIDPRICE
橡胶指数 T3
橡胶指数 TEMA
橡胶指数 WMA
橡胶指数 ADX
橡胶指数 AROON
沪铜指数 MA
沪铜指数 DEMA
沪铜指数 EMA
沪铜指数 KAMA
沪铜指数 MIDPRICE
沪铜指数 T3
沪铜指数 TEMA
沪铜指数 WMA
沪铜指数 ADX
沪铜指数 AROON
沪锌指数 MA
沪锌指数 DEMA
沪锌指数 EMA
沪锌指数 KAMA
沪锌指数 MIDPRICE
沪锌指数 T3
沪锌指数 TEMA
沪锌指数 WMA
沪锌指数 ADX
沪锌指数 AROON
白糖指数 MA
白糖指数 DEMA
白糖指数 EMA
白糖指数 KAMA
白糖指数 MIDPRICE
白糖指数 T3
白糖指数 TEMA
白糖指数 WMA
白糖指数 ADX
白糖指数 AROON
螺纹钢指数 MA
螺纹钢指数 DEMA
螺纹钢指数 EMA
螺纹钢指数 KAMA
螺纹钢指数 MIDPRICE
螺纹钢指数 T3
螺纹钢指数 TEMA
螺纹钢指数 WMA
螺纹钢指数 ADX
螺纹钢指数 AROON
豆油指数 MA
豆油指数 DEMA
豆油指数 EMA
豆油指数 KAMA
豆油指数 MIDPRICE
豆油指数 T3
豆油指数 TEMA
豆油指数 WMA
豆油指数 ADX
豆油指数 AROON
豆粕指数 MA
豆粕指数 DEMA
豆粕指数 EMA
豆粕指数 KAMA
豆粕指数 MIDPRICE
豆粕指数 T3
豆粕指数 TEMA
豆粕指数 WMA
豆粕指数 ADX
豆粕指数 AROON
铁矿石指数 MA
铁矿石指数 DEMA
铁矿石指数 EMA
铁矿石指数 KAMA
铁矿石指数 MIDPRICE
铁矿石指数 T3
铁矿石指数 TEMA
铁矿石指数 WMA
铁矿石指数 ADX
铁矿

In [89]:
result

,MA,DEMA,EMA,KAMA,MIDPRICE,T3,TEMA,WMA,ADX,AROON
pta指数,0.652051,0.656789,0.619934,0.681217,0.525312,0.561400,0.273080,0.555425,-0.011227,-0.351174
塑料指数,0.216476,0.332222,0.196535,0.251691,0.188993,0.272298,0.256152,0.083029,-0.036772,-0.123028
橡胶指数,0.641973,0.688975,0.639991,0.510409,0.582691,0.697338,0.443793,0.786595,0.142018,0.321938
沪铜指数,0.547847,0.511142,0.625435,0.498196,0.552489,0.684203,0.404499,0.522198,0.296519,0.134609
沪锌指数,0.003510,0.095983,0.330636,0.380012,0.010305,0.270830,0.099061,0.023732,0.111371,0.071472
白糖指数,0.377157,0.194070,0.260986,0.196795,0.334602,0.289355,0.184144,0.377553,0.201794,0.404955
螺纹钢指数,0.520423,0.514305,0.500972,0.484139,0.622004,0.409054,0.517091,0.599577,0.147954,-0.115049
豆油指数,0.442908,0.289843,0.446734,0.164638,0.341139,0.382519,0.232211,0.305573,0.187066,0.166320
豆粕指数,0.479370,0.575025,0.347016,0.389230,0.541026,0.471407,0.355282,0.640632,0.366434,0.576826
铁矿石指数,0.307215,0.489441,-0.005460,0.216726,0.000858,0.060591,0.577567,0.526279,0.284390,0.479868


In [90]:
# homework3
df = handle_data(pd.read_excel(f'D:/data/raw_data/十种大宗商品指数文件/PTA指数.xlsx'))
df['20ma'] = df['close'].rolling(20).mean()
df['slope'] = talib.LINEARREG_SLOPE(df['20ma'],5)
df

,open,high,low,close,volume,20ma,slope
timestamp,,,,,,,
2006-12-18,8833.27,8833.27,8833.27,8833.27,49396.0,NaN,NaN
2006-12-19,8867.51,8867.51,8867.51,8867.51,49906.0,NaN,NaN
2006-12-20,8919.04,8919.04,8919.04,8919.04,26180.0,NaN,NaN
2006-12-21,8870.84,8870.84,8870.84,8870.84,19754.0,NaN,NaN
2006-12-22,8880.97,8880.97,8880.97,8880.97,10300.0,NaN,NaN
...,...,...,...,...,...,...,...
2023-02-24,5584.70,5639.20,5567.92,5631.04,2013593.0,5585.2770,-5.82840
2023-02-27,5625.04,5652.81,5527.11,5536.46,2361800.0,5571.8200,-9.52105
2023-02-28,5540.54,5617.59,5511.59,5608.79,2386902.0,5565.5255,-10.07315


In [78]:
# homework 4
df = handle_data(pd.read_excel(f'D:/data/raw_data/十种大宗商品指数文件/PTA指数.xlsx'))
df['ATR'] = talib.ATR(df.high,df.low,df.close,14)
'''
elif (data_price['position'][i - 1] == 1) and (
         data_price['close'][i] < price_in - parameter * data_price['ATR'][i]
    data_price['flag'][i] = -1  # 记录买入还是卖出，1是买入
    data_price['position'][i] = 0  # 仓位记录为0，表示没有仓位了
    date_out = data_price.index[i]  # 记录卖出的时间 年-月-日
    price_out = data_price['close'][i]  # 记录卖出的价格，这里是以收盘价卖出
    Sell.append([date_out, price_out, '回落平仓'])  # 把卖出记录保存到Sell列表里
'''